In [1]:
import pandas as pd
import numpy as np
from openpyxl import load_workbook
from xlrd import open_workbook
import re
import os
import time
from os import listdir
from os.path import isfile, join
from ddf_utils.str import to_concept_id
from ddf_utils.index import create_datapackage

In [11]:
df1 = pd.read_excel('source/WPP2015_POP_F12_A_CHILD_DEPENDENCY_RATIO_1564.XLS',sheetname= 'ESTIMATES', skiprows=16, na_values='…')
xl = pd.ExcelFile('source/WPP2015_POP_F12_A_CHILD_DEPENDENCY_RATIO_1564.XLS')

In [12]:
xl.sheet_names

['ESTIMATES',
 'MEDIUM VARIANT',
 'HIGH VARIANT',
 'LOW VARIANT',
 'CONSTANT-FERTILITY',
 'INSTANT-REPLACEMENT',
 'ZERO-MIGRATION',
 'CONSTANT-MORTALITY',
 'NO CHANGE',
 'NOTES']

In [23]:
df = xl.parse('ESTIMATES', na_values='…')

df = df.ix[14:]
df.columns = df.iloc[0]

In [25]:
df.columns.values

array(['Index', 'Variant', 'Major area, region, country or area *',
       'Notes', 'Country code', '1950', 1955.0, 1960.0, 1965.0, 1970.0,
       1975.0, 1980.0, 1985.0, 1990.0, 1995.0, 2000.0, 2005.0, 2010.0,
       2015.0], dtype=object)

In [26]:
df.iloc[0][5:18]

14
1950      1950
1955.0    1955
1960.0    1960
1965.0    1965
1970.0    1970
1975.0    1975
1980.0    1980
1985.0    1985
1990.0    1990
1995.0    1995
2000.0    2000
2005.0    2005
2010.0    2010
Name: 14, dtype: object

In [19]:
df.head(20)

14,Index,Variant,"Major area, region, country or area *",Notes,Country code,1950,1955.0,1960.0,1965.0,1970.0,1975.0,1980.0,1985.0,1990.0,1995.0,2000.0,2005.0,2010.0,2015.0
14,Index,Variant,"Major area, region, country or area *",Notes,Country code,1950,1955.000000,1960.000000,1965.000000,1970.000000,1975.000000,1980.000000,1985.000000,1990.000000,1995.000000,2000.000000,2005.000000,2010.000000,2015.000000
15,1,Estimates,WORLD,NaN,900,56.7132,59.916041,64.102481,66.480872,65.843947,64.190122,60.263902,56.174949,53.889492,51.680522,47.860498,43.354561,40.643193,39.701890
16,2,Estimates,More developed regions,a,901,42.3319,42.496493,44.669929,43.098996,40.420813,37.143379,34.161377,31.974514,30.624503,29.259139,26.984651,24.951913,24.288150,24.779339
17,3,Estimates,Less developed regions,b,902,64.2773,68.877684,73.721256,77.674124,77.132584,75.321372,70.065149,64.452623,61.096698,58.044925,53.337516,47.793366,44.306483,42.783279
18,4,Estimates,Least developed countries,c,941,74.3153,75.816318,78.163650,81.272397,83.656482,85.115449,85.891824,86.245132,85.612318,83.145654,80.268532,77.570505,74.882056,70.822178
19,5,Estimates,"Less developed regions, excluding least develo...",d,934,63.0573,68.018806,73.158195,77.212531,76.308574,74.111161,68.146426,61.850610,58.140021,54.870201,49.783614,43.709200,39.907229,38.419652
20,6,Estimates,"Less developed regions, excluding China",NaN,948,68.3542,70.701479,74.987384,79.061047,79.038352,77.037207,74.349434,71.988307,69.072652,65.130145,60.214378,55.798940,52.219666,49.509192
21,7,Estimates,High-income countries,e,1503,44.0262,44.628725,46.822928,45.607136,43.257492,39.819597,36.544127,34.200554,32.681620,31.274936,29.086913,26.996009,25.905964,25.972749
22,8,Estimates,Middle-income countries,e,1517,62.1223,66.791592,72.012452,76.039816,75.287361,73.583115,68.290946,62.444181,58.986168,55.748712,50.659616,44.691414,41.027493,39.524188
23,9,Estimates,Upper-middle-income countries,e,1502,59.7267,66.785402,72.548210,76.285630,74.686722,72.781731,64.405233,54.752512,50.224363,47.052758,41.637417,33.948141,29.985233,29.454883


In [15]:
df1.head(20)

,Index,Variant,"Major area, region, country or area *",Notes,Country code,1950,1955,1960,1965,1970,1975,1980,1985,1990,1995,2000,2005,2010,2015
0,1,Estimates,WORLD,NaN,900,56.713202,59.916041,64.102481,66.480872,65.843947,64.190122,60.263902,56.174949,53.889492,51.680522,47.860498,43.354561,40.643193,39.701890
1,2,Estimates,More developed regions,a,901,42.331943,42.496493,44.669929,43.098996,40.420813,37.143379,34.161377,31.974514,30.624503,29.259139,26.984651,24.951913,24.288150,24.779339
2,3,Estimates,Less developed regions,b,902,64.277297,68.877684,73.721256,77.674124,77.132584,75.321372,70.065149,64.452623,61.096698,58.044925,53.337516,47.793366,44.306483,42.783279
3,4,Estimates,Least developed countries,c,941,74.315267,75.816318,78.163650,81.272397,83.656482,85.115449,85.891824,86.245132,85.612318,83.145654,80.268532,77.570505,74.882056,70.822178
4,5,Estimates,"Less developed regions, excluding least develo...",d,934,63.057332,68.018806,73.158195,77.212531,76.308574,74.111161,68.146426,61.850610,58.140021,54.870201,49.783614,43.709200,39.907229,38.419652
5,6,Estimates,"Less developed regions, excluding China",NaN,948,68.354202,70.701479,74.987384,79.061047,79.038352,77.037207,74.349434,71.988307,69.072652,65.130145,60.214378,55.798940,52.219666,49.509192
6,7,Estimates,High-income countries,e,1503,44.026180,44.628725,46.822928,45.607136,43.257492,39.819597,36.544127,34.200554,32.681620,31.274936,29.086913,26.996009,25.905964,25.972749
7,8,Estimates,Middle-income countries,e,1517,62.122251,66.791592,72.012452,76.039816,75.287361,73.583115,68.290946,62.444181,58.986168,55.748712,50.659616,44.691414,41.027493,39.524188
8,9,Estimates,Upper-middle-income countries,e,1502,59.726701,66.785402,72.548210,76.285630,74.686722,72.781731,64.405233,54.752512,50.224363,47.052758,41.637417,33.948141,29.985233,29.454883
9,10,Estimates,Lower-middle-income countries,e,1501,64.758306,66.798279,71.445275,75.780573,75.935374,74.447281,72.508040,70.923366,68.610498,64.953601,59.820273,55.401799,51.634318,48.665809


In [4]:
df = pd.read_excel('source/metadata.xlsx', parse_cols = "A:E")
df['name'] = ''
df['description'] = ''
df['sourceurl'] = 'https://esa.un.org/unpd/wpp/Download/Standard/Population/'

In [20]:
df.loc[df['Indicator'] == 'SexRatio_MalePer100Female']

,FileName,TypeBY,SEX,Indicator,Directory,name,description,sourceurl
2,WPP2015_POP_F04_SEX_RATIO_OF_TOTAL_POPULATION.XLS,Year,na,SexRatio_MalePer100Female,ddf--datapoints--SexRatio_MalePer100Female--by...,,,https://esa.un.org/unpd/wpp/Download/Standard/...
44,WPP2015_POP_F10_1_SEX_RATIO_BY_BROAD_AGE_GROUP...,Age,na,SexRatio_MalePer100Female,ddf--datapoints--SexRatio_MalePer100Female--by...,,,https://esa.un.org/unpd/wpp/Download/Standard/...


In [26]:
df.loc[(df['TypeBY'] == 'Age') & (df['Indicator'] == 'SexRatio_MalePer100Female')]

,FileName,TypeBY,SEX,Indicator,Directory,name,description,sourceurl
44,WPP2015_POP_F10_1_SEX_RATIO_BY_BROAD_AGE_GROUP...,Age,na,SexRatio_MalePer100Female,ddf--datapoints--SexRatio_MalePer100Female--by...,test,,https://esa.un.org/unpd/wpp/Download/Standard/...


In [24]:
df.loc[(df['TypeBY'] == 'Age') & (df['Indicator'] == 'SexRatio_MalePer100Female'), 'name'] = 'test'

In [6]:
df

,FileName,TypeBY,SEX,Indicator,Directory,name,description,sourceurl
0,WPP2015_POP_F01_1_TOTAL_POPULATION_BOTH_SEXES.XLS,Year,Total,Population,ddf--datapoints--Population--by--ref_area_code...,,,https://esa.un.org/unpd/wpp/Download/Standard/...
1,WPP2015_POP_F01_2_TOTAL_POPULATION_MALE.XLS;WP...,Year,male,Population,ddf--datapoints--Population--by--ref_area_code...,,,https://esa.un.org/unpd/wpp/Download/Standard/...
2,WPP2015_POP_F04_SEX_RATIO_OF_TOTAL_POPULATION.XLS,Year,na,SexRatio_MalePer100Female,ddf--datapoints--SexRatio_MalePer100Female--by...,,,https://esa.un.org/unpd/wpp/Download/Standard/...
3,WPP2015_POP_F05_MEDIAN_AGE.XLS,Year,na,MedianAgeOfPopulation,ddf--datapoints--MedianAgeOfPopulation--by--re...,,,https://esa.un.org/unpd/wpp/Download/Standard/...
4,WPP2015_POP_F06_POPULATION_DENSITY.XLS,Year,na,PopulationDensityPerSqKm,ddf--datapoints--PopulationDensityPerSqKm--by-...,,,https://esa.un.org/unpd/wpp/Download/Standard/...
5,WPP2015_POP_F11_A_TOTAL_DEPENDENCY_RATIO_1564.XLS,Year,na,DependencyRatio1564,ddf--datapoints--DependencyRatio1564--by--ref_...,,,https://esa.un.org/unpd/wpp/Download/Standard/...
6,WPP2015_POP_F11_B_TOTAL_DEPENDENCY_RATIO_2064.XLS,Year,na,DependencyRatio2064,ddf--datapoints--DependencyRatio2064--by--ref_...,,,https://esa.un.org/unpd/wpp/Download/Standard/...
7,WPP2015_POP_F11_C_TOTAL_DEPENDENCY_RATIO_2069.XLS,Year,na,DependencyRatio2069,ddf--datapoints--DependencyRatio2069--by--ref_...,,,https://esa.un.org/unpd/wpp/Download/Standard/...
8,WPP2015_POP_F11_D_TOTAL_DEPENDENCY_RATIO_2564.XLS,Year,na,DependencyRatio2564,ddf--datapoints--DependencyRatio2564--by--ref_...,,,https://esa.un.org/unpd/wpp/Download/Standard/...
9,WPP2015_POP_F11_E_TOTAL_DEPENDENCY_RATIO_2569.XLS,Year,na,DependencyRatio2569,ddf--datapoints--DependencyRatio2569--by--ref_...,,,https://esa.un.org/unpd/wpp/Download/Standard/...
